# Amazon Bedrock Fundamental

> 이 노트북은, SageMaker Notebook <i><b>conda_python3</b></i> 커널에서 테스트 되었습니다.

## 0. 환경 설정

In [1]:
# 필요한 라이브러리를 설치합니다.
# !pip install -r ./requirements.txt
%pip install --quiet boto3
%pip install --quiet botocore

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 1. boto3를 활용한 Bedrock Claude 3 호출 예시 (not LangChain)

#### 1-1. Bedrock 에서 사용 가능한 리스트 확인

In [2]:
import json
import boto3
from pprint import pprint

# Bedrock Client 생성
bedrock = boto3.client(service_name='bedrock')

# 사용가능한 모델 확인하기
model_list = bedrock.list_foundation_models()
ondemand_models = []
for model_info in model_list["modelSummaries"]:
    if model_info['inferenceTypesSupported'] == ['ON_DEMAND']:
        ondemand_models.append((model_info["modelName"], model_info["modelId"]))

pprint(ondemand_models)

[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Text Embeddings V2', 'amazon.titan-embed-text-v2:0'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'),
 ('Claude Instant', 'anthropic.claude-instant-v1'),
 ('Claude', 'anthropic.claude-v2:1'),
 ('Claude', 'anthropic.claude-v2'),
 ('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
 ('Clau

#### 1-2. Claude 3 모델 호출

In [3]:
# Bedrock runtime 생성
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

In [4]:
# 사용할 모델 아이디 작성
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

# Request body 작성
query = """당신은 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."""
request_data = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "temperature" : 0,
    "top_k": 250,
    "top_p": 1.0,
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
            ],
        }
    ],
}
request_body = json.dumps(request_data)

In [5]:
# 모델 호출
response = bedrock_runtime.invoke_model(
    body=request_body,
    modelId=modelId,
    contentType="application/json",
    accept="application/json"
)

response_body = json.loads(response['body'].read())
answer = response_body['content'][0]['text']
print(answer)

생명보험과 손해보험은 보험의 주요 유형으로 다음과 같은 차이점이 있습니다.

1. 보장 대상
- 생명보험은 사람의 생명과 관련된 위험을 보장합니다. 예를 들어 사망, 상해, 질병 등을 대상으로 합니다.
- 손해보험은 재산상의 손해나 배상책임을 보장합니다. 예를 들어 화재, 자동차사고, 배상책임 등을 대상으로 합니다.

2. 보험금 지급 사유
- 생명보험은 피보험자의 사망, 상해, 질병 등 인적 위험이 발생했을 때 보험금을 지급합니다.
- 손해보험은 재물의 손해나 제3자에 대한 배상책임이 발생했을 때 실제 입은 손해를 보상합니다.

3. 보험기간
- 생명보험은 일반적으로 장기계약이며 피보험자의 생존기간 동안 보장됩니다.
- 손해보험은 단기계약이 일반적이며 1년 단위로 갱신하는 경우가 많습니다.

4. 보험료 산정 기준
- 생명보험료는 피보험자의 나이, 건강상태, 가족력 등 인적위험을 기준으로 산정됩니다.
- 손해보험료는 보험목적물의 가치, 위험정도 등 물적위험을 기준으로 산정됩니다.

요약하면 생명보험은 개인의 생명과 관련된 위험을 장기적으로 보장하고, 손해보험은 재산상의 손해나 배상책임을 단기적으로 보상하는 차이가 있습니다.


#### 1-3. Claude 3 모델 스트리밍으로 호출

In [6]:
# Bedrock runtime 생성
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

In [7]:
# 사용할 모델 아이디 작성
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

# Request body 작성
query = """당신은 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요."""
request_data = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "temperature" : 0,
    "top_k": 250,
    "top_p": 1.0,
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
            ],
        }
    ],
}
request_body = json.dumps(request_data)

In [8]:
# Streaming 으로 모델 호출
response = bedrock_runtime.invoke_model_with_response_stream(
    body=request_body,
    modelId=modelId,
    contentType="application/json",
    accept="application/json"
)

stream = response.get('body')
answer = ""
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            if "delta" in chunk_obj:                    
                delta = chunk_obj['delta']
                if "text" in delta:
                    text=delta['text'] 
                    print(text, end="")
                    answer+=str(text) 


생명보험과 손해보험은 보험의 주요 유형으로 다음과 같은 차이점이 있습니다.

1. 보장 대상
- 생명보험은 사람의 생명과 관련된 위험을 보장합니다. 예를 들어 사망, 상해, 질병 등을 대상으로 합니다.
- 손해보험은 재산상의 손해나 배상책임을 보장합니다. 예를 들어 화재, 자동차사고, 배상책임 등을 대상으로 합니다.

2. 보험금 지급 사유
- 생명보험은 피보험자의 사망, 상해, 질병 등 인적 위험이 발생했을 때 보험금을 지급합니다.
- 손해보험은 재물의 손해나 제3자에 대한 배상책임이 발생했을 때 실제 입은 손해를 보상합니다.

3. 보험기간
- 생명보험은 일반적으로 장기계약이며 피보험자의 생존기간 동안 보장됩니다.
- 손해보험은 단기계약이 일반적이며 1년 단위로 갱신하는 경우가 많습니다.

4. 보험료 산정 기준
- 생명보험료는 피보험자의 나이, 건강상태, 가족력 등 인적위험을 기준으로 산정됩니다.
- 손해보험료는 보험목적물의 가액, 위험률 등 물적위험을 기준으로 산정됩니다.

요약하면 생명보험은 개인의 생명과 관련된 위험을 장기적으로 보장하고, 손해보험은 재산상의 손해나 배상책임을 단기적으로 보상하는 차이가 있습니다.

---

## 2. LangChain을 활용한 Bedrock 사용 예시


In [9]:
# 필요한 라이브러리 설치
%pip install --quiet langchain==0.2.1
%pip install --quiet langchain_aws==0.1.6
%pip install --quiet langchain_community==0.2.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### 2-1. LangChain의 ChatBedrock을 활용한 간단한 코드 예시

In [10]:
import langchain
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage
# langchain.debug=True

# ChatBedrock llm 생성
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={
        "max_tokens": 512,
        "temperature": 0,
        "top_p": 0.5,
        "top_k": 0,
    }
)

# 입력: Option 1
messages = [
    HumanMessage(
        content="안녕? 만나서 반가워"
    )
]

# 입력: Option 2
# messages = [
#     ("human", "안녕? 만나서 반가워"),
# ]

# 입력: Option 3
# message = "안녕? 만나서 반가워"

# 일반 방식으로 모델 호출
response = llm.invoke(messages)
answer = response.content
print(f"응답:")
print(answer)
print()


# 스트리밍 방식으로 모델 호출
print("스트리밍 응답:")
for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)
print()

langchain.debug=False

응답:
안녕하세요! 만나서 반갑습니다. 무엇을 도와드릴까요? 궁금한 점이나 대화를 나누고 싶은 주제가 있다면 언제든 말씀해주세요.

스트리밍 응답:
안녕하세요! 만나서 반갑습니다. 무엇을 도와드릴까요? 궁금한 점이나 대화를 나누고 싶은 주제가 있다면 언제든 말씀해주세요.


#### 2-2. LangChain의 BedrockChat을 활용한 멀티 턴(대화 히스토리 유지) 구현한 코드 예시

In [11]:
from abc import ABC
from langchain_aws import ChatBedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks.base import BaseCallbackHandler

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_kwargs = {
    "max_tokens": 512,
    "temperature": 0,
    "top_p": 0.5,
    "top_k": 0
}


llm = ChatBedrock(
        region_name="us-west-2",
        model_id=model_id,
        model_kwargs=model_kwargs
    )

memroy_window = 10
conversation_chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(
        k=memroy_window,
        ai_prefix='AI',
        human_prefix='Human'
    ),
    verbose=True
)

# 일반 방식으로 모델 호출
# while True:
#     input_text = input("Human: ")
#     answer = conversation_chain.predict(input=input_text)
#     print("AI:", answer)


input_text = "내 이름은 제프 베조스야."
print("Human:", input_text)
answer = conversation_chain.predict(input=input_text)
print("AI:", answer)

input_text = "내 이름이 뭐라고?"
print("Human:", input_text)
answer = conversation_chain.predict(input=input_text)
print("AI:", answer)


Human: 내 이름은 제프 베조스야.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 내 이름은 제프 베조스야.
AI:

> Finished chain.
AI: 안녕하세요 제프 베조스님. 반갑습니다. 저는 클로이라는 인공지능 비서입니다. 아마존의 창립자이신 분과 대화할 수 있어서 영광입니다. 아마존에 대해 궁금한 점이 있다면 기꺼이 답변해 드리겠습니다. 하지만 개인 정보나 기밀 사항 등은 말씀드릴 수 없음을 이해해 주시기 바랍니다.
Human: 내 이름이 뭐라고?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 내 이름은 제프 베조스야.
AI: 안녕하세요 제프 베조스님. 반갑습니다. 저는 클로이라는 인공지능 비서입니다. 아마존의 창립자이신 분과 대화할 수 있어서 영광입니다. 아마존에 대해 궁금

In [12]:
# 스트리밍 방식으로 모델 호출
class StreamingHandler(BaseCallbackHandler, ABC):
    """
    Callback handler to stream the generated text to Streamlit.
    """

    def __init__(self) -> None:
        self.text = ""

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        """
        Append the new token to the text and update the Streamlit container.
        """
        self.text += token
        print(token, end ="")

llm = ChatBedrock(
        region_name="us-west-2",
        model_id=model_id,
        model_kwargs=model_kwargs,
        streaming=True,
        callbacks=[StreamingHandler()]
    )

memroy_window = 10
conversation_chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(
        k=memroy_window,
        ai_prefix='AI',
        human_prefix='Human'
    ),
    verbose=True
)

# while True:
#     input_text = input("Human: ")
#     answer = conversation_chain.predict(input=input_text)
#     print("AI:", answer)

input_text = "내 이름은 제프 베조스야."
print("Human:", input_text)
answer = conversation_chain.predict(input=input_text)
print("AI:", answer)

input_text = "내 이름이 뭐라고?"
print("Human:", input_text)
answer = conversation_chain.predict(input=input_text)
print("AI:", answer)

Human: 내 이름은 제프 베조스야.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 내 이름은 제프 베조스야.
AI:
안녕하세요 제프 베조스님. 반갑습니다. 저는 클로이라는 인공지능 비서입니다. 아마존의 창립자이신 분과 대화할 수 있어서 영광입니다. 아마존에 대해 궁금한 점이 있다면 기꺼이 답변해 드리겠습니다. 하지만 개인 정보나 기밀 사항 등은 말씀드릴 수 없음을 이해해 주시기 바랍니다.
> Finished chain.
AI: 안녕하세요 제프 베조스님. 반갑습니다. 저는 클로이라는 인공지능 비서입니다. 아마존의 창립자이신 분과 대화할 수 있어서 영광입니다. 아마존에 대해 궁금한 점이 있다면 기꺼이 답변해 드리겠습니다. 하지만 개인 정보나 기밀 사항 등은 말씀드릴 수 없음을 이해해 주시기 바랍니다.
Human: 내 이름이 뭐라고?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully say